### Binary Intrusion Detection — Model Training & Comparison

This notebook performs binary classification (Attack vs Benign) on the engineered CIC-IIoT dataset `combined_engineered_features.csv`.
Primary objective: train K-Nearest Neighbors (KNN).  
Extended objective: train and compare multiple classifiers (KNN, Random Forest, SVM, Logistic Regression) with hyperparameter tuning, compute full set of metrics, plot ROC & confusion matrices, save best models and metric summary.

**Notes:**
- Input dataset path: `data/features/combined_engineered_features.csv`
- Output models saved to: `data/models/`
- Metric summary saved to: `data/models/metrics_summary.csv`

#### Imports & Config

In [17]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix
)
import joblib
sns.set(style="whitegrid")
%matplotlib inline

# Paths
FEATURE_PATH = os.path.join("data", "features", "combined_engineered_features.csv")
MODELS_DIR = os.path.join("data", "models")
os.makedirs(MODELS_DIR, exist_ok=True)

#### Load & Prepare

In [18]:
# Cell: load data
df = pd.read_csv(FEATURE_PATH)
print("Loaded dataset:", df.shape)

# Ensure label1 exists
assert "label1" in df.columns, "label1 column not found in dataset!"

# Prepare features and binary target
X = df.drop(columns=["label1","label2","label3","label4","label_full"], errors='ignore')
y = df["label1"].apply(lambda x: 1 if str(x).lower() == "attack" else 0)

print("Feature matrix shape:", X.shape)
print("Class distribution:\n", y.value_counts().rename({0:"Benign",1:"Attack"}))


Loaded dataset: (685671, 45)
Feature matrix shape: (685671, 40)
Class distribution:
 label1
Benign    400672
Attack    284999
Name: count, dtype: int64


#### Train/Test Split

In [19]:
RANDOM_STATE = 42

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)


Train shape: (548536, 40) Test shape: (137135, 40)


#### Evaluate model (metrics + plots)

In [20]:
# Cell: helper functions
def compute_metrics(y_true, y_pred, y_prob=None):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)  # sensitivity
    f1 = f1_score(y_true, y_pred, zero_division=0)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    spec = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    roc_auc = roc_auc_score(y_true, y_prob) if (y_prob is not None) else np.nan
    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "specificity": spec,
        "f1": f1,
        "roc_auc": roc_auc,
        "confusion_matrix": cm
    }

def plot_confusion(cm, labels=["Benign","Attack"], title="Confusion Matrix"):
    plt.figure(figsize=(4,3))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Reds", xticklabels=labels, yticklabels=labels)
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

def plot_roc(y_true, y_prob, label):
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    auc = roc_auc_score(y_true, y_prob)
    plt.plot(fpr, tpr, label=f"{label} (AUC={auc:.3f})")
    return auc


#### Define Models + Parameter Grids

In [21]:
model_defs = {}

model_defs["knn"] = {
    "estimator": KNeighborsClassifier(),
    "param_grid": {
        "n_neighbors": [3, 5, 7, 9, 11],
        "weights": ["uniform", "distance"],
        "metric": ["euclidean", "manhattan", "minkowski"]
    }
}

model_defs["nb"] = {
    "estimator": GaussianNB(),
    "param_grid": {
        "var_smoothing": np.logspace(-10, -6, 5)
    }
}

MODELS_TO_RUN = ["knn", "nb"]

print(f"Models to run: {MODELS_TO_RUN}")


Models to run: ['knn', 'nb']


In [22]:
def train_and_evaluate_model(model_key, model_def, X_train, X_test, y_train, y_test, base_dir, cv_folds=3, n_iter=5):
    """Train, evaluate and save results for a single model."""
    spec = model_def
    estimator = spec["estimator"]
    param_grid = spec["param_grid"]

    MODEL_DIR = os.path.join(base_dir, model_key)
    os.makedirs(MODEL_DIR, exist_ok=True)

    print(f"\n=== Training {model_key.upper()} ===")

    # --- Randomized SearchCV ---
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=RANDOM_STATE)
    search = RandomizedSearchCV(
        estimator,
        param_distributions=param_grid,
        n_iter=n_iter,
        scoring="f1",
        cv=cv,
        n_jobs=-1,
        random_state=RANDOM_STATE,
        verbose=1
    )
    search.fit(X_train, y_train)
    best = search.best_estimator_

    # --- Evaluate ---
    y_pred = best.predict(X_test)
    y_prob = best.predict_proba(X_test)[:, 1] if hasattr(best, "predict_proba") else best.decision_function(X_test)
    metrics = compute_metrics(y_test, y_pred, y_prob)

    # --- Save model ---
    model_path = os.path.join(MODEL_DIR, f"{model_key}_best_model.pkl")
    joblib.dump(best, model_path)

    # --- Save confusion matrix ---
    cm = metrics["confusion_matrix"]
    plt.figure(figsize=(4,3))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Reds",
                xticklabels=["Benign","Attack"], yticklabels=["Benign","Attack"])
    plt.title(f"{model_key.upper()} Confusion Matrix")
    plt.xlabel("Predicted"); plt.ylabel("True")
    cm_path = os.path.join(MODEL_DIR, f"{model_key}_confusion_matrix.png")
    plt.tight_layout(); plt.savefig(cm_path, dpi=300); plt.close()

    # --- Save ROC curve ---
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    roc_auc = roc_auc_score(y_test, y_prob)
    plt.figure(figsize=(5,4))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f"AUC={roc_auc:.3f}")
    plt.plot([0,1],[0,1],"k--", lw=1)
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
    plt.title(f"ROC - {model_key.upper()}"); plt.legend(loc="lower right")
    roc_path = os.path.join(MODEL_DIR, f"{model_key}_roc_curve.png")
    plt.tight_layout(); plt.savefig(roc_path, dpi=300); plt.close()

    print(f"Saved model, confusion matrix, and ROC for {model_key}")

    # --- Return summary row ---
    return {
        "model": model_key,
        "best_params": search.best_params_,
        "cv_f1": search.best_score_,
        "test_accuracy": metrics["accuracy"],
        "test_precision": metrics["precision"],
        "test_recall": metrics["recall"],
        "test_specificity": metrics["specificity"],
        "test_f1": metrics["f1"],
        "test_roc_auc": metrics["roc_auc"]
    }

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_DIR = os.path.join("results", f"model_results_{timestamp}")
os.makedirs(RUN_DIR, exist_ok=True)
print(f"Results will be saved under: {RUN_DIR}")

results = []

for key in MODELS_TO_RUN:
    result_row = train_and_evaluate_model(
        model_key=key,
        model_def=model_defs[key],
        X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,
        base_dir=RUN_DIR,
        cv_folds=2,
        n_iter=1
    )
    results.append(result_row)

metrics_df = pd.DataFrame(results)
metrics_csv = os.path.join(RUN_DIR, "metrics_summary_all_models.csv")
metrics_df.to_csv(metrics_csv, index=False)
print(f"\nMetrics summary saved: {metrics_csv}")
display(metrics_df)

Results will be saved under: results\model_results_20251024_175610

=== Training KNN ===


ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.